In [16]:
import numpy as np
from scipy.linalg import solve

In [20]:
np.random.seed(0)

### 3. Next step solution

In [21]:
def interior_point_step(A, b, x, y, s, mu):
    n = x.shape[0]  # variables
    e = np.ones_like(x)

    # Diagonal matrices
    X = np.diag(x)
    S_inv = np.diag(1 / s)

    # Short-step update for mu
    mu_prime = (1 - (1 / (6 * np.sqrt(n)))) * mu

    # Solve (A S^{-1} X A^T) k = b - mu' * A S^{-1} e
    M = A @ S_inv @ X @ A.T
    rhs = b - mu_prime * A @ S_inv @ e
    dy = np.linalg.solve(M, rhs) 

    ds = -A.T @ dy
    dx = -X @ S_inv @ ds + mu_prime * S_inv @ e - x

    # Update
    x_prime = x + dx
    y_prime = y + dy
    s_prime = s + ds

    return x_prime, y_prime, s_prime, mu_prime


In [18]:
np.random.seed(0)
m, n = 2, 4
A = np.random.rand(m, n)
b = np.random.rand(m)
c = np.random.rand(n)

# Strictly feasible starting point
x0 = np.random.rand(n) + 1.0
y0 = np.random.rand(m)
s0 = np.random.rand(n) + 1.0
mu0 = (x0 @ s0) / n

In [22]:
interior_point_step(A, b, x0, y0, s0, mu0)

(array([ 0.73163932,  0.53785387,  1.02813365, -0.81165922]),
 array([ 2.02530982, -2.01437071]),
 array([2.51614654, 2.77021386, 1.97191022, 3.6731912 ]),
 np.float64(2.0233470621836656))

### 6. Show that vectors are strictly feasiblefor X and its dual

In [ ]:
c = np.array([-3, -4, 0, 0, 0])

A = np.array([[3, 3, 3, 0, 0],
              [3, 1, 0, 1, 0],
              [1, 4, 0, 0, 1]])

b = np.array([4, 3, 4])

In [30]:
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15])
y = np.array([-4/5, -4/5, -2/3])
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3])

In [51]:
#  x > 0
print("X is strictily positive:", all(x > 0)) 

# Ax = b
print("Ax = b:", np.allclose(A @ x, b, atol=1e-10))

# s > 0
print("s is strictly positive:", all(s > 0))

# A^T y + s = c
print("A^T y < c:", all(A.T @ y < c))
print("A^T y + s = c:", np.allclose(A.T @ y + s, c, atol=1e-10))

print("x, y and s are strictly feasible solutions of both problem X and its dual")

X is strictily positive: True
Ax = b: True
s is strictly positive: True
A^T y < c: True
A^T y + s = c: True
x, y and s are strictly feasible solutions of both problem X and its dual
